# Step 1: Formulate the query to the DB

In [25]:
import numpy as np
# Step 1: Get the id's of the subset of LatLonPairs via a DB Query
objs_list = c3.HycomLatLongPair.fetch(
    spec={"include": "id, lat, lon", "filter": "lat > 24 && lat < 25 && lon > -95 && lon < -94", "order": "j,i"}).objs



ids_list = [obj.id for obj in objs_list]
lat_list = [obj.lat for obj in objs_list]
lon_list = [obj.lon for obj in objs_list]

# create the x, y grids
lat_grid = np.unique(np.array(lat_list))
lon_grid = np.unique(np.array(lon_list))
# ids_list


In [26]:
print(len(ids_list))

576


# Step 2: Query the DB

In [29]:
# Step 2: Query with Metadata
my_metric = c3.SimpleMetric(id = "AverageWaterU_HycomLatLongPair",
                            name = "AverageWaterU",
                            description = "Calculates average surface X-velocity for the given interval",
                            srcType = "HycomLatLongPair",
                            path = "surfaceHindcastData",
                            expression = "avg(avg(normalized.data.water_u))"
                           )

my_metric.toJson()
# Note: the above metric can also be specified in the /seed folder in the tags repo to be broadly available

# Step 3: Create EvalMetricsSpec
my_spec = c3.EvalMetricsSpec(
            ids = ids_list[0:10],
            expressions = ["AverageWaterU"],
            start = "2021-09-02T23:00:00",
            end = "2021-09-15T00:00:00",
            interval = "HOUR", 
        )

# Query the server with Metadata
evalMetricsResult = c3.HycomLatLongPair.evalMetricsWithMetadata(spec=my_spec,
                                                      overrideMetrics=[my_metric])



In [30]:
evalMetricsResult.result

c3.Mapp<string, map<string,Timeseries>>({'GOMu0.04_76-148': c3.Mapp<string, Timeseries>({'AverageWaterU': c3.NormTimeseriesDouble(
                                       m_start=datetime.datetime(2021, 9, 2, 23, 0),
                                       m_end=datetime.datetime(2021, 9, 15, 0, 0),
                                       m_data=c3.Arry<double>([-0.01900000125169754,
                                                0.015000000596046448,
                                                0.04800000041723251,
                                                0.07300000637769699,
                                                0.09000000357627869,
                                                0.10100000351667404,
                                                0.10000000149011612,
                                                0.09000000357627869,
                                                0.07500000298023224,
                                                0.0600000023841

In [ ]:
# # Step 2 Alternative: Query without Metadata

# # Query without Metadata using the metric defined in the seed directory
# evalMetricsResult_wo_Meta = c3.HycomLatLongPair.evalMetrics(spec=c3.EvalMetricsSpec(
#             ids = ids_list,
#             expressions = ["TestAverageWaterU"], # does this refer to the one above?
#             start = "2021-09-23T23:00:00",
#             end = "2021-09-24T00:00:00",
#             interval = "HOUR", 
#         ))

In [ ]:
# # inspect the results
# evalMetricsResult.result

# Step 3: sort into a (lat, lon, time) array

In [ ]:
# Step 3: Sort the results back into a (lat,lon,time) array for use in the simulator
# create a sorted list out of the data in order to get the matrix shape back
data_list = []
for single_id in ids_list:
    data_list.append(evalMetricsResult.result[single_id]["AverageWaterU"].m_data)

# reshape into (T, lat, lon)
data_array = np.array(data_list)
data_array = data_array.reshape(-1, len(lat_grid), len(lon_grid))

# As full function

In [ ]:
# def getDataSubset_dictionary(lon_min, lon_max, lat_min, lat_max, metric, start, end, batchSize=100):
# we can run filter with i,j => might be faster because integer comparisons faster than float
lat_lon_filter = "lon>"+str(lon_min)+" && lon<"+str(lon_max)+" && lat>"+str(lat_min)+" && lat<"+str(lat_max)

# get the lat-lon grid
objs_list = c3.HycomLatLongPair.fetch(
    spec={"include": "id, lat, lon", "filter": lat_lon_filter, "order": "j,i"}).objs

ids_list = [obj.id for obj in objs_list]
lat_list = [obj.lat for obj in objs_list]
lon_list = [obj.lon for obj in objs_list]
# create the x, y grids
lat_grid = np.unique(np.array(lat_list))
lon_grid = np.unique(np.array(lon_list))

my_spec = c3.EvalMetricsSpec(
#             filter = lat_lon_filter,
        ids = ids_list,
        limit = -1,
        expressions = [metric],
        start = start.strftime("%Y-%m-%dT%H:%M:%S"),
        end = end.strftime("%Y-%m-%dT%H:%M:%S"),
        interval = "HOUR"
    )


# Query the metric on DB
evalMetricsResult = c3.HycomLatLongPair.evalMetrics(spec=my_spec)

# 
data_list = []
for single_id in ids_list:
    data_list.append(evalMetricsResult.result[single_id][metric].m_data)

# reshape into (T, lat, lon)
data_array = np.array(data_list)
data_array = data_array.reshape(-1, len(lat_grid), len(lon_grid))

#     return data_array, lat_grid, lon_grid

In [ ]:
data_list

In [ ]:
from datetime import datetime
start = datetime(2021,9,23,23)
end = datetime(2021,9,24,0)

In [ ]:
lon_min=-98
lon_max=-97
lat_min=18
lat_max=19
metric="TestAverageWaterU"

In [ ]:
# 18 && lat < 19 && lon > -98 && lon < -97
data_array, lat_grid, lon_grid = getDataSubset_dictionary(lon_min=-98, lon_max=-97, lat_min=18, lat_max=19,
                         metric="TestAverageWaterU",
                         start=start,
                         end=end)

In [ ]:
data_array.shape

# Full Function 2: full loading function from Darren

In [13]:
def getDataSubset(imin,jmin,imax,jmax,metric,start,end,batchSize=1):
    """Returns a subset for the given metric data from the given start and end time.
 
    Args:
        imin (int): Minimum longitude from lat-long pair indices
        jmin (int): Minimum latitude from lat-long pair indices
        imax (int): Maximum longitude from lat-long pair indices
        jmax (int): Maximum latitude from lat-long pair indices
        metric (str): Metric to be extracted from the dataset
        start (datetime): Start time of the subset
        end (datetime): End time of the subset
        batchSize (int): Number of lat-long pairs to be extracted in each batch
 
    Returns:
        array: Numpy array of the subsetted data ordered as [time,lat,lon]
 
    Notes:
        - Currently limited to hour resolution
        - Only handles a single metric
        - The data indexing is not efficient since the indices must be parsed from the
        lat-long pair ids.  If the HycomLatLong pair table was ordered by i,j, then
        the pairs would get extracted in i,j order and there would be no need to parse.
        this could be done by defining a new id that is a simple ascending integer.
 
    """
   # we can run filter with i,j => might be faster because integer comparisons faster than float
    my_spec = c3.EvalMetricsSpec(
            filter = "i>="+str(imin)+" && i<="+str(imax)+" && j>="+str(jmin)+" && j<="+str(jmax),
            limit = 10,
            expressions = [metric],
            start = start.strftime("%Y-%m-%d"),
            end = end.strftime("%Y-%m-%d"),
            interval = "HOUR"
        )
    # Evaluate the Spec using EvalSourceSpec which returns a stream of numpy arrays
    sourceType = c3.TypeRef(typeName="HycomLatLongPair")
    # creates dumped np.array files
    em_source_spec = c3.EvalMetricsSourceSpec.createNdArraySourceSpec(my_spec, sourceType)
    #em_source_spec.batchExport()
    stream_spec = c3.BatchStreamSpec(batchSize=batchSize)
    source_stream = em_source_spec.toStream(stream_spec)
    # we can get u and v together, multi-metrics.
   
    # Process the data in to a (time,long,lat) data array
    duration = end - start
    duration_in_s = duration.total_seconds()
    nt = int(divmod(duration_in_s, 3600)[0]) # total duration in hours
    data = np.zeros((nt,imax-imin+1,jmax-jmin+1))

    
    print("begin streaming")
    while source_stream.hasNext(): 
        print("begin streaming 2")
        stream = source_stream.next()
        print(type(stream))
        idi = -1
        # Transpose time series
        for id in stream.indices[0]:
            idi += 1
            istr,jstr = id.split('_')[1].split('-')
            i = int(istr) - imin
            j = int(jstr) - jmin
            ti = 0
            for t in stream.indices[2]:
                data[ti,i,j] = stream.data[idi,0,ti]
                ti += 1
   
    # Cleans up any persisted files storing snapshot of data source
    em_source_spec.cleanUp()
    # Removes spec from database
    em_source_spec.remove()
    return data

In [14]:
from datetime import datetime 
import numpy as np
udata = getDataSubset(
    imin = 81,
    jmin = 81,
    imax = 90,
    jmax = 90,
    metric = "TestAverageWaterU",
    start = datetime(2021,10,1),
    end = datetime(2021,10,8)
)
udata.shape

begin streaming


KeyboardInterrupt: 

# Full Process (created by Conor)


In [1]:
import numpy as np
from datetime import datetime
import time

In [15]:
def getDataSubset(t_interval, lat_interval, lon_interval, metric="TestAverageWaterU", interval="HOUR"):
    """
    Returns a subset for the given metric data from the given start and end time. 
    This version does not using Streaming and so may be slower (?)
 
    Args:
        t_interval (float): [t_0, t_T] in POSIX time where t_0 and t_T are the start and end timestamps respectively
        lat_interval (float tuple): [y_lower, y_upper] in degrees
        lon_interval (float tuple): [x_lower, x_upper] in degrees
        
        metric (str): Metric to be extracted from the dataset   
        interval (string): frequency of datapoints to output (only works for "HOUR" right now!)
 
    Returns:
        array: Numpy array of the subsetted data ordered as [time,lat,lon]
 
    Notes:
        - Currently designed/tested for hour resolution
        - Only tested for single metric
        - Comments included with times for different sections of the function. These times are not averaged 
          (only one run used) and are there to present an idea of the runtime of different sections of the code.
          NOTE: significnat time variance when fetching from c3
    """
    pre_query_start = time.time()
    ###PRE-QUERY SECTION###
    #takes 0.1423 seconds for 1x1 lat-lon, 1 day
    #convert times to datetime
    start_time = datetime.fromtimestamp(t_interval[0])
    end_time = datetime.fromtimestamp(t_interval[1])

    #filter for query
    filter = "lat>={} && lat<={} && lon>={} && lon<={}".format(lat_interval[0], lat_interval[1], 
                                                               lon_interval[0], lon_interval[1])

    #get lat, lon dimensions
    objs_list = c3.HycomLatLongPair.fetch(spec={"include": "id, lat, lon", "filter": filter, "limit": -1}).objs
    lat_dim =  len(np.unique([obj.lat for obj in objs_list]))
    lon_dim = len(np.unique([obj.lon for obj in objs_list]))
    
    print("pre_query_time: ", time.time() - pre_query_start)
    
    query_start = time.time()

    ###QUERY SECTION###
    #takes 17.17 seconds for 1x1 lat-lon, 1 day
    # Query the server for EvalMetrics data
    my_spec = c3.EvalMetricsSpec(
                filter = filter,
                limit = -1,
                expressions = [metric],
                start = start_time.strftime("%Y-%m-%d"),
                end = end_time.strftime("%Y-%m-%d"),
                interval = interval
            )

    evalMetricsResult = c3.HycomLatLongPair.evalMetrics(spec=my_spec)
    
    print("query_time: ", time.time() - query_start)
    
    post_query_start = time.time()
    
    ###POST_QUERY SECTION###
    #takes 0.02296 seconds for 1x1 lat-lon, 1 day
    
    #calculate number of discrete time points 
    #(currently calculates number of hours, need to modify to work for different intervals)
    duration = end_time - start_time
    duration_in_s = duration.total_seconds()
    num_times = int(divmod(duration_in_s, 3600)[0])

    #extract data into array
    keys = sorted([key for key in evalMetricsResult.result])
    arr = np.zeros(shape=(num_times,len(keys)))
    for i in range(len(keys)):
        arr[:, i] = np.array(evalMetricsResult.result[keys[i]]["TestAverageWaterU"].m_data)

    arr = np.swapaxes(arr.reshape(num_times, lon_dim, lat_dim), 1, 2)
    
    print("post_query_time: ", time.time() - post_query_start)
    
    return arr

In [ ]:
data = getDataSubset(t_interval=(datetime(2021,9,2,12).timestamp(), datetime(2021,9,3,12).timestamp()), 
                     lat_interval=(23, 27),
                     lon_interval=(-91, -87))

pre_query_time:  1.6925451755523682


In [2]:
def getDataSubset_latlong(t_interval, lat_interval, lon_interval, 
                          metric="TestAverageWaterU", batchSize=100, interval="HOUR"):
    """Returns a subset for the given metric data from the given start and end time. 
    Conor alterted Darren's function to use lat-long pairs as input + other changes
    
    TODO: get same arguments as get_current_data_subset_from_c3 (in Data_subsetting_example.ipynb)
    
    and for now HOUR time interval
 
    Args:
        t_interval (float): [t_0, t_T] in POSIX time where t_0 and t_T are the start and end timestamps respectively
        lat_interval (float tuple): [y_lower, y_upper] in degrees
        lon_interval (float tuple): [x_lower, x_upper] in degrees
        
        metric (str): Metric to be extracted from the dataset   
        batchSize (int): Number of lat-long pairs to be extracted in each batch
        interval (string): frequency of datapoints to output (only works for "HOUR" right now!)
 
    Returns:
        array: Numpy array of the subsetted data ordered as [time,lat,lon]
 
    Notes:
        - Currently limited to hour resolution
        - Only handles a single metric
        - The data indexing is not efficient since the indices must be parsed from the
        lat-long pair ids.  If the HycomLatLong pair table was ordered by i,j, then
        the pairs would get extracted in i,j order and there would be no need to parse.
        this could be done by defining a new id that is a simple ascending integer.
 
    """
    start_time = datetime.fromtimestamp(t_interval[0])
    end_time = datetime.fromtimestamp(t_interval[1])
   # we can run filter with i,j => might be faster because integer comparisons faster than float
    my_spec = c3.EvalMetricsSpec(
            filter = "lat>={} && lat<={} && lon>={} && lon<={}".format(lat_interval[0], lat_interval[1], 
                                                                       lon_interval[0], lon_interval[1]),
            limit = 10,
            expressions = [metric],
            start = start_time.strftime("%Y-%m-%d"),
            end = end_time.strftime("%Y-%m-%d"),
            interval = interval
        )
    # Evaluate the Spec using EvalSourceSpec which returns a stream of numpy arrays
    sourceType = c3.TypeRef(typeName="HycomLatLongPair")
    # creates dumped np.array files
    em_source_spec = c3.EvalMetricsSourceSpec.createNdArraySourceSpec(my_spec, sourceType)
#     em_source_spec.batchExport()
    stream_spec = c3.BatchStreamSpec(batchSize=batchSize)
    source_stream = em_source_spec.toStream(stream_spec)
    # we can get u and v together, multi-metrics.
   
    # Process the data in to a (time,long,lat) data array
    duration = end_time - start_time
    duration_in_s = duration.total_seconds()
    nt = int(divmod(duration_in_s, 3600)[0]) # total duration in hours
    
#     data = np.zeros((nt,imax-imin+1,jmax-jmin+1))
    
    
    # How can we get the results back in order to easy reshape them?
    # -> Darren tries it out.
    #TODO: see how to stream to make sure it's in order (final array shape should be (time, y, x) or (time, lat, lon))
    while source_stream.hasNext(): 
        stream = source_stream.next()
        idi = -1
        
        # Transpose time series
        for id in stream.indices[0]:
#             idi += 1
#             istr,jstr = id.split('_')[1].split('-')
#             i = int(istr) - imin
#             j = int(jstr) - jmin
#             ti = 0
            for t in stream.indices[2]:
#                 data[ti,i,j] = stream.data[idi,0,ti]
#                 ti += 1
                print(stream.data)
    
    source_stream.close()
   
    # Cleans up any persisted files storing snapshot of data source
    em_source_spec.cleanUp()
    # Removes spec from database
    em_source_spec.remove()
    return data

In [ ]:
t_interval=(datetime(2021,9,2).timestamp(), datetime(2021,9,3).timestamp())
lat_interval=(24, 25)
lon_interval=(-90, -89) 
metric="TestAverageWaterU"
batchSize=100
interval="HOUR"

start_time = datetime.fromtimestamp(t_interval[0])
end_time = datetime.fromtimestamp(t_interval[1])
# we can run filter with i,j => might be faster because integer comparisons faster than float
my_spec = c3.EvalMetricsSpec(
        filter = "lat>={} && lat<={} && lon>={} && lon<={}".format(lat_interval[0], lat_interval[1], 
                                                                   lon_interval[0], lon_interval[1]),
        limit = -1,
        expressions = [metric],
        start = start_time.strftime("%Y-%m-%d"),
        end = end_time.strftime("%Y-%m-%d"),
        interval = interval
    )
print("created spec")
# Evaluate the Spec using EvalSourceSpec which returns a stream of numpy arrays
sourceType = c3.TypeRef(typeName="HycomLatLongPair")
# creates dumped np.array files
em_source_spec = c3.EvalMetricsSourceSpec.createNdArraySourceSpec(my_spec, sourceType)
# em_source_spec.batchExport()
stream_spec = c3.BatchStreamSpec(batchSize=batchSize)
source_stream = em_source_spec.toStream(stream_spec)

print("created stream")
# we can get u and v together, multi-metrics.

# Process the data in to a (time,long,lat) data array
duration = end_time - start_time
duration_in_s = duration.total_seconds()
nt = int(divmod(duration_in_s, 3600)[0]) # total duration in hours

# data = np.zeros((nt,imax-imin+1,jmax-jmin+1))


# How can we get the results back in order to easy reshape them?
# -> Darren tries it out.
#TODO: see how to stream to make sure it's in order (final array shape should be (time, y, x) or (time, lat, lon))
print("starting stream")
print(source_stream)
print(source_stream.next())
count = 0
while source_stream.hasNext() and count <= 10: 
    stream = source_stream.next()
    print(stream)
    idi = -1

    # Transpose time series
    for id in stream.indices[0]:
#             idi += 1
#             istr,jstr = id.split('_')[1].split('-')
#             i = int(istr) - imin
#             j = int(jstr) - jmin
#             ti = 0
        for t in stream.indices[2]:
#                 data[ti,i,j] = stream.data[idi,0,ti]
#                 ti += 1
            print(stream.data)
    count += 1

# Cleans up any persisted files storing snapshot of data source
em_source_spec.cleanUp()
# Removes spec from database
em_source_spec.remove()
u_data = data

created spec
created stream
starting stream
c3.EvalMetricsNdArrayStream(
 streamSpec=c3.BatchStreamSpec(batchSize=100, fileFetchTotalMemoryMb=512),
 dataSourceSpec=c3.EvalMetricsNdArraySourceSpec(
                  start=datetime.datetime(2021, 9, 2, 0, 0),
                  end=datetime.datetime(2021, 9, 3, 0, 0),
                  timeZone='NONE',
                  interval='HOUR',
                  cache=False,
                  filter='lat>=24 && lat<=25 && lon>=-90 && lon<=-89',
                  limit=-1,
                  expressions={'TestAverageWaterU'},
                  continueOnError=False,
                  id='f357230e-03bf-4411-abb3-0c8a96237861',
                  meta=c3.Meta(
                         tenantTagId=150,
                         tenant='dev',
                         tag='tc01d',
                         created=datetime.datetime(2021, 11, 9, 22, 44, 22, tzinfo=datetime.timezone.utc),
                         createdBy='con-mart@berkeley.edu',
          

In [ ]:
udata = getDataSubset_latlong(
    t_interval=(datetime(2021,9,2).timestamp(), datetime(2021,9,3).timestamp()),
    lat_interval=(24, 25),
    lon_interval=(-90, -89)   
)

In [ ]:
# Do a number of comparisons (i.e. small medium large lat-lon size, interpolation or not)
#iff i, j leads to efficiency improvement -> figure out how to convert lat-long to i,j?